<a href="https://colab.research.google.com/github/tluxxx/CandleStickStudies/blob/main/stock_data_check_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
import csv
import os

from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
from datetime import date, datetime, timedelta

import yfinance as yf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. colleting stick data from various sources

In [46]:
tickers = ['ADS.DE', 'AIR.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BEI.DE', 'BMW.DE', 'BNR.DE', 'CBK.DE', 'CON.DE', 'DTG.DE', 'DBK.DE']

In [3]:
PATH_XETRA = Path('/content/drive/My Drive/ColabNotebooks/06_BacktestBasics/02_stock_data_test/stock_data_xetra')
PATH_ONVISTA = Path('/content/drive/My Drive/ColabNotebooks/06_BacktestBasics/02_stock_data_test/stock_data_onvista')

In [47]:
# Processing XETRA Data, previously downloaded from Boerse Frankfurt)
all_tickers_df = []
ticker_info = []

for i, ticker in enumerate(tickers):
    file_name = f'Download Kurshistorie ({i}).csv'   # naming convention of the downloaded file {i=position in tickers}
    file_path = PATH_XETRA / file_name

    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        first_line = next(reader)
        if len(first_line) > 1:
            exchange = first_line[0].strip()
            wkn = first_line[1].strip()
            ticker_info.append({'Ticker': ticker, 'Exchange': exchange, 'WKN': wkn})
        else:
            print(f"Could not extract exchange and WKN from the first line of {file_name}")

        # Load and pre-process stock data
        column_names = ['date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Pieces']
        df = pd.read_csv(file_path, delimiter=';', skiprows=2, header=None, names=column_names, on_bad_lines='skip')
        columns_to_convert = ['Open', 'Close', 'High', 'Low', 'Volume', 'Pieces']
        for col in columns_to_convert:   # conversion from German format to pandas standard format
            df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
            df[col] = pd.to_numeric(df[col])

        df['date'] = pd.to_datetime(df['date'], format='%d.%m.%y')
        df.set_index('date', inplace=True)
        df.sort_index(ascending=True, inplace=True)

        df_selected = df[['Open', 'Close', 'High', 'Low']]
        df_selected.columns = pd.MultiIndex.from_product([['Open', 'Close', 'High', 'Low'], [ticker]])
        all_tickers_df.append(df_selected)

# Combine dataframes and create info table
xetra_df = pd.concat(all_tickers_df, axis=1)
xetra_info_df = pd.DataFrame(ticker_info)   # verify the name of the exchange and the ISIN-number )just for check
display(xetra_df.head(3))
# display(xetra_info_df)

,Open,Close,High,Low,Open,Close,High,Low,Open,Close,...,High,Low,Open,Close,High,Low,Open,Close,High,Low
,ADS.DE,ADS.DE,ADS.DE,ADS.DE,AIR.DE,AIR.DE,AIR.DE,AIR.DE,ALV.DE,ALV.DE,...,CON.DE,CON.DE,DTG.DE,DTG.DE,DTG.DE,DTG.DE,DBK.DE,DBK.DE,DBK.DE,DBK.DE
date,,,,,,,,,,,,,,,,,,,,,
2024-08-12,215.5,212.4,216.8,211.3,133.60,133.72,135.28,133.36,257.4,255.9,...,59.04,58.26,33.38,33.00,33.57,33.00,13.378,13.308,13.410,13.218
2024-08-13,213.2,213.3,213.7,208.9,134.72,134.26,134.82,133.58,256.3,256.9,...,59.28,57.90,33.21,33.25,33.52,33.12,13.386,13.300,13.386,13.142
2024-08-14,214.6,217.2,218.3,214.1,135.22,134.56,135.24,134.18,257.8,259.0,...,59.34,58.44,33.49,33.55,33.64,33.27,13.366,13.380,13.450,13.288


In [48]:
# Processing XETRA Data, previously downloaded from Onvista)
all_tickers_onvista_df = []
files_in_onvista = os.listdir(PATH_ONVISTA)

for ticker in tickers:
    # find for ticker matching csv-file
    ticker_prefix = ticker.split('.')[0] if '.' in ticker else ticker  # Extract the relevant part of the ticker for matching
    matching_file = None
    for file_name in files_in_onvista:
        if file_name.startswith(ticker_prefix + '_'):
            matching_file = file_name
            break

    # loading an pre-priocessing matching file
    if matching_file:
        file_path = PATH_ONVISTA / matching_file

        try:
            df_onvista = pd.read_csv(file_path, delimiter=';')
            df_onvista.rename(columns={'Datum': 'date'}, inplace=True)
            df_onvista.set_index('date', inplace=True)
            df_onvista.drop('Währung', axis=1, inplace=True)
            columns_to_convert = ['Eröffnung', 'Hoch', 'Tief', 'Schluss', 'Volumen']
            for col in columns_to_convert:
                df_onvista[col] = df_onvista[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
                df_onvista[col] = pd.to_numeric(df_onvista[col])
            df_onvista.rename(columns={'Eröffnung': 'Open', 'Hoch': 'High', 'Tief': 'Low', 'Schluss': 'Close', 'Volumen': 'Volume'}, inplace=True)

            # Create MultiIndex columns with price information and ticker symbol
            df_onvista.columns = pd.MultiIndex.from_product([['Open', 'High', 'Low', 'Close', 'Volume'], [ticker]])
            all_tickers_onvista_df.append(df_onvista)

        except FileNotFoundError:
            print(f"File not found for ticker {ticker}: {file_path}")
        except Exception as e:
            print(f"Error processing file for ticker {ticker}: {e}")
    else:
        print(f"No matching file found for ticker {ticker} in the ONVISTA directory.")

# Concatenate all dataframes into a single MultiIndex dataframe
onvista_df = pd.concat(all_tickers_onvista_df, axis=1)
onvista_df.index = pd.to_datetime(onvista_df.index, format='%d.%m.%Y')
display(onvista_df.head(3))

,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,...,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume
,ADS.DE,ADS.DE,ADS.DE,ADS.DE,ADS.DE,AIR.DE,AIR.DE,AIR.DE,AIR.DE,AIR.DE,...,DTG.DE,DTG.DE,DTG.DE,DTG.DE,DTG.DE,DBK.DE,DBK.DE,DBK.DE,DBK.DE,DBK.DE
date,,,,,,,,,,,,,,,,,,,,,
2020-08-12,262.2,264.6,258.5,263.7,537268,75.50,76.00,74.22,74.85,236522,...,NaN,NaN,NaN,NaN,NaN,8.230,8.393,8.211,8.260,8677919
2020-08-13,264.3,270.9,263.8,268.2,570422,73.71,74.81,72.42,73.25,175499,...,NaN,NaN,NaN,NaN,NaN,8.190,8.301,8.054,8.073,8000949
2020-08-14,267.1,267.1,262.5,265.9,512540,73.00,73.00,70.80,71.85,132720,...,NaN,NaN,NaN,NaN,NaN,8.079,8.109,7.838,8.069,8433006


In [49]:
# Downloading stocke data from yfinance
start_date = '2020-08-12'
end_date = '2025-08-12'

yfinance_df = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)
display(yfinance_df.head(3))

[*********************100%***********************]  12 of 12 completed


Price            Close                                               \
Ticker          ADS.DE     AIR.DE      ALV.DE     BAS.DE    BAYN.DE   
Date                                                                  
2020-08-12  251.997284  70.896317  146.022644  37.665539  51.405476   
2020-08-13  256.297607  69.380829  143.921616  37.306957  51.121571   
2020-08-14  254.099625  68.054771  142.698608  36.809334  50.536003   

Price                                                             ...  Volume  \
Ticker         BEI.DE     BMW.DE     BNR.DE    CBK.DE     CON.DE  ...  ALV.DE   
Date                                                              ...           
2020-08-12  93.783195  44.043179  47.553257  4.573765  72.048683  ...  981631   
2020-08-13  93.301254  43.601780  47.273430  4.461235  71.923058  ...  736348   
2020-08-14  92.202461  43.556889  46.171604  4.448214  72.865273  ...  712098   

Price                                                                   \
Ticker       BAS.DE  BAYN.DE  BEI.DE   BMW.DE  BNR.DE   CBK.DE  CON.DE   
Date                                                                     
2020-08-12  2324507  2543424  435115  1663294  347753  9727552  720448   
2020-08-13  1634725  1645233  214749  1025918  204611  6497552  568765   
2020-08-14  1872189  2180232  230306  1131446  169800  6298085  539970   

Price                       
Ticker       DBK.DE DTG.DE  
Date                        
2020-08-12  8677919    NaN  
2020-08-13  8000949    NaN  
2020-08-14  8433006    NaN  

[3 rows x 60 columns]

In [50]:
# Downloading stock data from Stooq
start_date_stooq = '20200812' # Stooq uses YYYYMMDD format
end_date_stooq = '20250812'

all_tickers_stooq_df = []

for ticker in tickers:
    stooq_url = f"https://stooq.com/q/d/l/?s={ticker.lower()}&d1={start_date_stooq}&d2={end_date_stooq}&i=d"

    try:
        # download and procss the dataframe
        df_stooq_single = pd.read_csv(stooq_url, sep=',')
        df_stooq_single['Date'] = pd.to_datetime(df_stooq_single['Date'])
        df_stooq_single.set_index('Date', inplace=True)
        # Create MultiIndex columns with price information and ticker symbol
        df_stooq_single.columns = pd.MultiIndex.from_product([['Open', 'High', 'Low', 'Close', 'Volume'], [ticker]])
        all_tickers_stooq_df.append(df_stooq_single)

    except Exception as e:
        print(f"Error downloading data for {ticker} from Stooq: {e}")
        print(f"Attempted URL: {stooq_url}")

# generating one dataframe
stooq_df = pd.concat(all_tickers_stooq_df, axis=1)
display(stooq_df)

,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,...,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume
,ADS.DE,ADS.DE,ADS.DE,ADS.DE,ADS.DE,AIR.DE,AIR.DE,AIR.DE,AIR.DE,AIR.DE,...,DTG.DE,DTG.DE,DTG.DE,DTG.DE,DTG.DE,DBK.DE,DBK.DE,DBK.DE,DBK.DE,DBK.DE
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-12,262.2,264.600,258.500,263.70,537268,75.500,76.000,74.22,74.85,236522,...,NaN,NaN,NaN,NaN,NaN,8.230,8.393,8.211,8.260,8677919
2020-08-13,264.3,270.900,263.800,268.20,570422,73.710,74.810,72.42,73.25,175499,...,NaN,NaN,NaN,NaN,NaN,8.190,8.301,8.054,8.073,8000949
2020-08-14,267.1,267.100,262.500,265.90,512540,73.000,73.000,70.80,71.85,132720,...,NaN,NaN,NaN,NaN,NaN,8.079,8.109,7.838,8.069,8433006
2020-08-17,266.6,268.600,264.800,268.60,361128,71.710,72.540,70.62,71.52,107028,...,NaN,NaN,NaN,NaN,NaN,8.029,8.100,7.936,7.963,5276006
2020-08-18,266.9,270.700,265.700,267.50,423762,70.800,71.500,69.19,69.21,183972,...,NaN,NaN,NaN,NaN,NaN,7.900,8.056,7.844,7.918,6443252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-06,167.0,167.000,162.649,162.90,825239,177.340,178.120,175.36,177.92,264810,...,39.97,40.33,39.65,39.80,790359.0,29.400,29.950,29.340,29.780,4865881
2025-08-07,162.3,168.800,160.750,166.55,923290,178.560,179.100,177.28,177.42,319316,...,39.80,41.18,39.56,41.06,1056409.0,29.830,30.750,29.830,30.565,6001952


In [ ]:
# Downloading stocke data from yfinance
start_date = '2020-08-12'
end_date = '2025-08-12'

yfinance_df = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True, 'stooq')
display(yfinance_df.head(3))

### 2. Plotting for Overview

In [8]:
# helper function for filtering MultiIndex-dataframes
def filter_and_flatten(seq_1, ticker, start, end):
    ''' helps to filter and flatten dataframes for plotting
    Args:
        seq_1 (pd.Dataframe): dataframe for processing (dates as row-index, Multilevel-Index (attributes, ticker) as columns)
        ticker (list of strings): tickers to be processed
        start (string): start datum for processing (YYYY-MM-DD)
        end (string): end datum for processing (YYYY-MM-DD)
    Returns:
        df (pd.Dataframe): filtered and flatteneed standrd dataframe return
    '''
    df = seq_1.loc[start:end, (['Open', 'Close', 'High', 'Low'], ticker)].copy()
    df.columns = df.columns.droplevel(1)
    return df

In [14]:
# plotting parameters
ticker = 'BAS.DE'
end_date = '2025-08-09'
end_date_plot = datetime.strptime(end_date, '%Y-%m-%d').date()
start_date_plot = end_date_plot - timedelta(days=6*30)

In [16]:
# filtering for ticker and flatten dataframes from different sources for plotting
xetra = filter_and_flatten(xetra_df, ticker, start_date_plot, end_date_plot)
onvista = filter_and_flatten(onvista_df, ticker, start_date_plot, end_date_plot)
yfinance = filter_and_flatten(yfinance_df, ticker, start_date_plot, end_date_plot)
stooq = filter_and_flatten(stooq_df, ticker, start_date_plot, end_date_plot)

# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('XETRA', 'onvista', 'yfinance', 'Stooq'),
                    horizontal_spacing=0.05,
                    vertical_spacing=0.1)

# Add candlestick traces
fig.add_trace(go.Candlestick(x=xetra.index,
                             open=xetra['Open'], high=xetra['High'], low=xetra['Low'], close=xetra['Close'],
                             name='XETRA'),
              row=1, col=1)

fig.add_trace(go.Candlestick(x=onvista.index,
                             open=onvista['Open'], high=onvista['High'], low=onvista['Low'], close=onvista['Close'],
                             name='ONVISTA'),
              row=1, col=2)

fig.add_trace(go.Candlestick(x=yfinance.index,
                             open=yfinance['Open'], high=yfinance['High'], low=yfinance['Low'], close=yfinance['Close'],
                             name='yfinance'),
              row=2, col=1)

fig.add_trace(go.Candlestick(x=stooq.index,
                             open=stooq['Open'], high=stooq['High'], low=stooq['Low'], close=stooq['Close'],
                             name='Stooq'),
              row=2, col=2)

# finetuning layout
fig.update_layout(title=f'Candlestick Plots for {ticker} (Last 6 Months) from Different Sources',
                  height=1000,
                  width=1500)

fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=1, col=2)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=2)

# Display the plot
fig.show()

In [17]:
# defining the comparison period
ticker = 'BAS.DE'
start_date_diff = '2024-08-11'
end_date_diff = '2025-08-09'

In [19]:
# calculate the differences between the data
xetra = filter_and_flatten(xetra_df, ticker, start_date_diff, end_date_diff)
onvista = filter_and_flatten(onvista_df, ticker, start_date_diff, end_date_diff)
yfinance = filter_and_flatten(yfinance_df, ticker, start_date_diff, end_date_diff)
stooq = filter_and_flatten(stooq_df, ticker, start_date_diff, end_date_diff)

diff_yfinance_xetra = yfinance - xetra
diff_stooq_xetra = stooq - xetra
diff_onvista_xetra = onvista - xetra

# prparation for tabulation of differences
attributes = ['Open', 'Close', 'High', 'Low']
diff_dfs = {}

for attribute in attributes:
    diff_dfs[f"{attribute}_diff_yfinance_xetra"] = diff_yfinance_xetra[attribute]
    diff_dfs[f"{attribute}_diff_stooq_xetra"]    = diff_stooq_xetra[attribute]
    diff_dfs[f"{attribute}_diff_onvista_xetra"]  = diff_onvista_xetra[attribute]

# Combine into a single DataFrame, indexing and display of basi statistics
all_differences_df = pd.DataFrame(diff_dfs)
all_differences_df.index = pd.to_datetime(all_differences_df.index).fillna(0)
display(all_differences_df.describe())


,Open_diff_yfinance_xetra,Open_diff_stooq_xetra,Open_diff_onvista_xetra,Close_diff_yfinance_xetra,Close_diff_stooq_xetra,Close_diff_onvista_xetra,High_diff_yfinance_xetra,High_diff_stooq_xetra,High_diff_onvista_xetra,Low_diff_yfinance_xetra,Low_diff_stooq_xetra,Low_diff_onvista_xetra
count,252.000000,252.000000,252.0,252.000000,252.000000,252.0,252.000000,252.000000,252.0,252.000000,252.000000,252.0
mean,-1.664818,0.000238,0.0,-1.665301,0.000218,0.0,-1.706054,0.000417,0.0,-1.623783,0.000298,0.0
std,1.049674,0.001157,0.0,1.063659,0.001024,0.0,1.072910,0.001522,0.0,1.046594,0.001185,0.0
min,-3.359696,0.000000,0.0,-3.611652,0.000000,0.0,-4.061652,0.000000,0.0,-2.683377,0.000000,0.0
25%,-2.336802,0.000000,0.0,-2.338893,0.000000,0.0,-2.368860,0.000000,0.0,-2.307213,0.000000,0.0
50%,-2.202354,0.000000,0.0,-2.202114,0.000000,0.0,-2.238342,0.000000,0.0,-2.179545,0.000000,0.0
75%,-0.000002,0.000000,0.0,-0.000002,0.000000,0.0,-0.000002,0.000000,0.0,-0.000002,0.000000,0.0
max,0.379998,0.010000,0.0,1.329998,0.005000,0.0,0.269998,0.010000,0.0,1.439998,0.005000,0.0


In [28]:
# plotting the differences
fig = make_subplots(rows=2, cols=2, subplot_titles=attributes,
                    vertical_spacing=0.1)
attribute_map = {attributes[0]: (1, 1), attributes[1]: (1, 2), attributes[2]: (2, 1), attributes[3]: (2, 2)}

# Define colors for consistency across subplots
yfinance_color = 'blue'
stooq_color = 'red'

for attribute in attributes:
    row, col = attribute_map[attribute]

    fig.add_trace(go.Scatter(x=diff_yfinance_xetra.index, y=diff_yfinance_xetra[attribute], # yfinance-xetra
                             mode='lines', name=f'yfinance - XETRA',
                             line=dict(color=yfinance_color),
                             showlegend=(row==1 and col==1)),
                  row=row, col=col)

    fig.add_trace(go.Scatter(x=diff_stooq_xetra.index, y=diff_stooq_xetra[attribute], #stooq - xetra
                             mode='lines', name=f'Stooq - XETRA',
                             line=dict(color=stooq_color),
                             showlegend=(row==1 and col==1)),
                  row=row, col=col)

# Update layout
fig.update_layout(title=f'Price Differences relative to XETRA for {ticker}',
                  height=600, width=1000,
                  margin=dict(l=50, r=50, t=100, b=50))

# Show the plot
fig.show()

In [30]:
def comparison_sets(set_1, set_2, tickers, start_date_diff, end_date_diff):
    prices = ['Open', 'Close', 'High', 'Low']
    all_tickers_results = []

    for ticker in tickers:
        ticker_results ={'Ticker': ticker}

        for price in prices:
            set_1_series = set_1.loc[start_date_diff:end_date_diff, (price, ticker)]
            set_2_series = set_2.loc[start_date_diff:end_date_diff, (price, ticker)]
            price_difference = abs(set_1_series - set_2_series)
            date_max_deviation = price_difference.idxmax()
            max_deviation = price_difference.max()
            ticker_results[f'{price}_max_dev'] = max_deviation
            ticker_results[f'{price}_dev_%'] = max_deviation/ set_1.loc[date_max_deviation,(price, ticker)]
            ticker_results[f'{price}_md_date'] = date_max_deviation

        all_tickers_results.append(ticker_results)

    results = pd.DataFrame(all_tickers_results)
    return results

In [31]:
# defining the comparison period
start_date_diff = '2024-08-11'
end_date_diff = '2025-08-09'

In [32]:
# comparing XETRA with yfinance
results = comparison_sets(xetra_df, yfinance_df, tickers, start_date_diff, end_date_diff)
display(results)

,Ticker,Open_max_dev,Open_dev_%,Open_md_date,Close_max_dev,Close_dev_%,Close_md_date,High_max_dev,High_dev_%,High_md_date,Low_max_dev,Low_dev_%,Low_md_date
0,ADS.DE,3.938351,0.015958,2025-02-26,6.015988,0.023381,2025-02-10,7.415988,0.028666,2025-02-10,4.250000,0.021384,2025-07-23
1,AIR.DE,8.083302,0.049597,2025-01-21,8.743302,0.053430,2025-01-20,9.443302,0.057462,2025-01-20,6.763302,0.041837,2025-01-21
2,ALV.DE,17.431592,0.049200,2025-03-24,16.939850,0.047853,2025-03-17,20.231592,0.056655,2025-03-24,15.371054,0.041353,2025-05-06
3,BAS.DE,3.359696,0.065491,2025-03-24,3.611652,0.076844,2025-01-24,4.061652,0.085599,2025-01-24,2.683377,0.050687,2025-03-18
4,BAYN.DE,1.094785,0.047870,2025-03-24,2.570000,0.115922,2025-05-14,0.824785,0.035643,2025-03-24,2.650000,0.119964,2025-05-14
5,BEI.DE,4.436569,0.036666,2025-04-15,4.232431,0.034721,2025-03-24,5.136569,0.042207,2025-04-15,4.232431,0.034721,2025-03-24
6,BMW.DE,6.093196,0.075039,2025-03-24,5.643737,0.067557,2025-03-17,7.282062,0.090236,2025-01-24,4.502062,0.057778,2025-01-24
7,BNR.DE,2.953084,0.045941,2025-03-24,2.848904,0.046444,2024-11-25,3.412668,0.058098,2025-04-29,2.379244,0.035437,2024-08-29
8,CBK.DE,0.867915,0.036964,2025-03-24,1.505380,0.062672,2025-04-29,1.525380,0.063452,2025-04-29,0.695380,0.029960,2025-04-29
9,CON.DE,3.927155,0.054925,2025-03-24,3.945279,0.061205,2025-04-15,4.565279,0.070149,2025-04-15,3.525279,0.055048,2025-04-15


In [ ]:
# comparing XETRA with stooq
results = comparison_sets(xetra_df, stooq_df, tickers, start_date_diff, end_date_diff)
display(results)

,Ticker,Open_max_dev,Open_dev_%,Open_md_date,High_max_dev,High_dev_%,High_md_date,Low_max_dev,Low_dev_%,Low_md_date,Close_max_dev,Close_dev_%,Close_md_date
0,ADS.DE,0.001,0.000005,2025-06-25,0.001,0.000006,2025-08-08,0.001,0.000005,2025-06-20,0.000,0.000000,2024-08-12
1,AIR.DE,0.360,0.002012,2025-08-07,0.001,0.000006,2025-05-27,0.001,0.000006,2025-05-27,0.001,0.000006,2025-06-12
2,ALV.DE,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.050,0.000146,2025-07-21,0.000,0.000000,2024-08-12
3,BAS.DE,0.010,0.000238,2025-06-06,0.010,0.000233,2025-06-27,0.005,0.000122,2024-08-14,0.005,0.000121,2024-08-14
4,BAYN.DE,0.010,0.000371,2025-06-30,0.005,0.000190,2024-08-15,0.005,0.000179,2024-08-21,0.005,0.000190,2024-08-15
5,BEI.DE,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12
6,BMW.DE,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12
7,BNR.DE,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12
8,CBK.DE,0.005,0.000404,2024-08-12,0.005,0.000398,2024-08-12,0.010,0.000361,2025-06-09,0.005,0.000406,2024-08-13
9,CON.DE,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12,0.000,0.000000,2024-08-12


In [ ]:
# Comparing XETRA with OneVista
results = comparison_sets(xetra_df, onvista_df, tickers, start_date_diff, end_date_diff)
display(results)

,Ticker,Open_max_dev,Open_dev_%,Open_md_date,High_max_dev,High_dev_%,High_md_date,Low_max_dev,Low_dev_%,Low_md_date,Close_max_dev,Close_dev_%,Close_md_date
0,ADS.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
1,AIR.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
2,ALV.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
3,BAS.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
4,BAYN.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
5,BEI.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
6,BMW.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
7,BNR.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
8,CBK.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12
9,CON.DE,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12,0.0,0.0,2024-08-12


In [38]:
# asessment of suspicious days in yfinance
def suspicious_day_detection(set_1, tickers, start_date_diff, end_date_diff, threshold):
    all_ticker_suspicious_days = []

    for ticker in tickers:
        set_1_df = set_1.loc[start_date_diff:end_date_diff, (['Open', 'Close', 'High', 'Low'], ticker)].copy()
        set_1_df.columns = set_1_df.columns.droplevel(1)
        set_1_df['range'] = (set_1_df['High'] - set_1_df['Low']) / set_1_df['Close']

        suspicious_days = set_1_df[set_1_df['range'] < threshold].index
        suspicious_days_list = suspicious_days.strftime('%Y-%m-%d').tolist()
        ticker_results = {'Ticker': ticker,
                          'nb. suspicious days': len(suspicious_days_list),
                          'suspicious dates': suspicious_days_list}
        all_ticker_suspicious_days.append(ticker_results)
    return pd.DataFrame(all_ticker_suspicious_days)

In [39]:
# suspicious days
threshold = 0.005 #  ( = 0.5%)
suspicious_days_df = suspicious_day_detection(yfinance_df, tickers, start_date_diff, end_date_diff, threshold)
display(suspicious_days_df)

,Ticker,nb. suspicious days,suspicious dates
0,ADS.DE,19,"[2024-11-01, 2024-11-18, 2024-11-28, 2024-12-1..."
1,AIR.DE,19,"[2024-11-15, 2024-11-18, 2024-11-28, 2024-12-1..."
2,ALV.DE,12,"[2024-11-01, 2024-11-28, 2024-12-10, 2025-01-2..."
3,BAS.DE,11,"[2024-11-01, 2024-11-28, 2024-12-10, 2025-01-2..."
4,BAYN.DE,10,"[2024-11-01, 2024-11-28, 2024-12-10, 2025-01-2..."
5,BEI.DE,18,"[2024-11-15, 2024-11-18, 2024-11-28, 2024-12-1..."
6,BMW.DE,14,"[2024-11-01, 2024-11-28, 2024-12-10, 2025-01-2..."
7,BNR.DE,24,"[2024-09-13, 2024-11-18, 2024-11-25, 2024-11-2..."
8,CBK.DE,13,"[2024-11-28, 2024-12-10, 2024-12-30, 2025-01-2..."
9,CON.DE,13,"[2024-11-28, 2024-12-10, 2025-01-20, 2025-01-2..."


In [40]:
# cross check suspicious days
ticker = 'BAS.DE'
susp_days = suspicious_days_df[suspicious_days_df['Ticker'] == ticker]['suspicious dates'].values[0]
data_susp_days= yfinance_df.loc[susp_days,(['Open', 'Close', 'High', 'Low'], ticker)]
data_susp_days

Price,Open,Close,High,Low
Ticker,BAS.DE,BAS.DE,BAS.DE,BAS.DE
Date,,,,
2024-11-01,42.382076,42.382076,42.382076,42.382076
2024-11-28,39.999298,39.999298,39.999298,39.999298
2024-12-10,42.847237,42.847237,42.847237,42.847237
2025-01-21,43.179501,43.179501,43.179501,43.179501
2025-01-24,43.388348,43.388348,43.388348,43.388348
2025-02-10,45.111351,45.111351,45.111351,45.111351
2025-03-17,49.905380,49.905380,49.905380,49.905380
2025-03-24,47.940304,47.940304,47.940304,47.940304


In [41]:
data_susp_days.index

DatetimeIndex(['2024-11-01', '2024-11-28', '2024-12-10', '2025-01-21',
               '2025-01-24', '2025-02-10', '2025-03-17', '2025-03-24',
               '2025-05-14', '2025-07-16', '2025-07-21'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [44]:
# comparison Xetra-Data with yfinance-data at suspicion days:
df_x= xetra_df.loc[data_susp_days.index,(['Open', 'Close', 'High', 'Low'], ticker)]
#df_x['range_%'] = (df_x['High'] - df_x['Close'])/ df_x['Close'] *100
df_x['yf_data'] = data_susp_days['Close']
df_x

,Open,Close,High,Low,yf_data
,BAS.DE,BAS.DE,BAS.DE,BAS.DE,
Date,,,,,
2024-11-01,44.460,45.555,45.555,44.375,42.382076
2024-11-28,42.295,42.060,42.505,41.805,39.999298
2024-12-10,44.700,44.930,45.125,44.420,42.847237
2025-01-21,45.055,45.610,45.630,44.940,43.179501
2025-01-24,45.995,47.000,47.450,45.945,43.388348
2025-02-10,47.500,47.315,47.780,47.245,45.111351
2025-03-17,52.930,52.470,53.040,52.110,49.905380
2025-03-24,51.300,50.020,51.300,50.020,47.940304


In [ ]:
# suspicious days
threshold = 0.005 #  ( = 0.5%)
suspicious_days_df = suspicious_day_detection(stooq_df, tickers, start_date_diff, end_date_diff, threshold)
display(suspicious_days_df)

,Ticker,nb. suspicious days,suspicious dates
0,ADS.DE,0,[]
1,AIR.DE,0,[]
2,ALV.DE,0,[]
3,BAS.DE,0,[]
4,BAYN.DE,0,[]
5,BEI.DE,0,[]
6,BMW.DE,0,[]
7,BNR.DE,0,[]
8,CBK.DE,0,[]
9,CON.DE,0,[]
